In [6]:
import numpy as np
import pandas as pd
import os
from shutil import copyfile
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.metrics.pairwise import euclidean_distances


import neuro_morpho_toolbox as nmt
%matplotlib inline

In [7]:
df = pd.read_excel('../Data/fMOST/fMOST External shipments tracker.xlsx', sheet_name=0, skiprows=[0], header=[1])
df = df[~df['Specimen ID'].isna()]
df.rename(columns={'AIBS Pedigree #':'CreLine'}, inplace=True)
df

,Specimen ID,CreLine,Tamoxifen Dose,Date brain shipped from AIBS,Shipment Tracking #,Counted in quarterly report?,Shipment Receipt @HUST Date,fMOST image priority,fMOST image date,fMOST image resolution (40x or 60x)/ Voxel Resolution,...,Data shipment notes (disk/tape IDs),Data upload to BIL (insert Date),Data conversion date,Data Conversion Format(s) and Bit Size,Re-conversion status,Reconstruction Notes,Reconstruction Link,Transfer to LIMS,R24 Archive transfer date,pixel resolution (microns per pixel)
0,281582-17049,Npr3-IR-CreERT2;Ai14;Ai140-281582,1-day TAM at full dose,2017-01-06,NaN,NaN,NaN,NaN,2017-03-29 00:00:00,40x,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,--,NaN
1,292790-17107,Etv1-CreERT2;Ai140-292790,1-day TAM at 1/5th dose,2017-02-27,NaN,NaN,NaN,NaN,2017-06-08 00:00:00,60x,...,NaN,NaN,NaT,Neurolucida,NaN,NaN,NaN,NaN,--,NaN
2,314107-17300,Plxnd1-CreER;Ai82;Ai140-314107,No TAM,2017-06-23,NaN,NaN,NaN,NaN,2017-09-09 00:00:00,60x,...,NaN,NaN,NaT,Vaa3D (8bit),NaN,NaN,NaN,Y,2018-06-29 00:00:00,NaN
3,228279-16128,Nkx2-1-CreERT2;Ai139-228279,1-day TAM at full dose,2016-03-12,NaN,NaN,NaN,NaN,2016-05-12 00:00:00,NaN,...,NaN,NaN,NaT,"Neurolucida, Vaa3D (8bit)",NaN,NaN,NaN,NaN,--,NaN
4,236174-16124,Gnb4-IRES2-CreERT2;Ai139-hyg-236174,1-day TAM at full dose,2016-03-12,NaN,NaN,NaN,NaN,2016-04-27 00:00:00,NaN,...,NaN,NaN,NaT,"Neurolucida, Vaa3D (8bit)",NaN,Done by AIBS and WMU,NaN,Y,2018-06-29 00:00:00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,462090-,Plxnd1-CreER;Ai166-462090,1-day TAM at 1/10th original dose at P35,2019-06-12,job0428,NaN,NaN,NaN,--,NaN,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,--,NaN
167,436253-,Tacr1-T2A-Cre;Sst-IRES-FlpO;Ai188-hyg-436253,No induction,2019-06-12,job0428,NaN,NaN,NaN,--,NaN,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,--,NaN
168,437264-,Tacr1-T2A-Cre;Sst-IRES-FlpO;Ai188-hyg-437264,No induction,2019-06-12,job0428,NaN,NaN,NaN,--,NaN,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,--,NaN
169,444771-,Tacr1-T2A-Cre;Sst-IRES-FlpO;Ai188-hyg-444771,No induction,2019-06-12,job0428,NaN,NaN,NaN,--,NaN,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,--,NaN


In [8]:
idlist = []
for i in df['Specimen ID'].tolist():
    tp = str(i).split('-')[-1].strip()
    if len(tp)==0:
        tp = str(i).split('-')[0].strip()
    idlist.append(tp) 
print(len(idlist))
iddict = dict(zip(idlist, idlist))
iddict['15257'] = '210254'
iddict['16124'] = '236174'
idlist = list(iddict.values())
print(len(idlist))

171
171


In [10]:
df.index = idlist

In [11]:
df.head()

,Specimen ID,CreLine,Tamoxifen Dose,Date brain shipped from AIBS,Shipment Tracking #,Counted in quarterly report?,Shipment Receipt @HUST Date,fMOST image priority,fMOST image date,fMOST image resolution (40x or 60x)/ Voxel Resolution,...,Data shipment notes (disk/tape IDs),Data upload to BIL (insert Date),Data conversion date,Data Conversion Format(s) and Bit Size,Re-conversion status,Reconstruction Notes,Reconstruction Link,Transfer to LIMS,R24 Archive transfer date,pixel resolution (microns per pixel)
17049,281582-17049,Npr3-IR-CreERT2;Ai14;Ai140-281582,1-day TAM at full dose,2017-01-06,NaN,NaN,NaN,NaN,2017-03-29 00:00:00,40x,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,--,NaN
17107,292790-17107,Etv1-CreERT2;Ai140-292790,1-day TAM at 1/5th dose,2017-02-27,NaN,NaN,NaN,NaN,2017-06-08 00:00:00,60x,...,NaN,NaN,NaT,Neurolucida,NaN,NaN,NaN,NaN,--,NaN
17300,314107-17300,Plxnd1-CreER;Ai82;Ai140-314107,No TAM,2017-06-23,NaN,NaN,NaN,NaN,2017-09-09 00:00:00,60x,...,NaN,NaN,NaT,Vaa3D (8bit),NaN,NaN,NaN,Y,2018-06-29 00:00:00,NaN
16128,228279-16128,Nkx2-1-CreERT2;Ai139-228279,1-day TAM at full dose,2016-03-12,NaN,NaN,NaN,NaN,2016-05-12 00:00:00,NaN,...,NaN,NaN,NaT,"Neurolucida, Vaa3D (8bit)",NaN,NaN,NaN,NaN,--,NaN
236174,236174-16124,Gnb4-IRES2-CreERT2;Ai139-hyg-236174,1-day TAM at full dose,2016-03-12,NaN,NaN,NaN,NaN,2016-04-27 00:00:00,NaN,...,NaN,NaN,NaT,"Neurolucida, Vaa3D (8bit)",NaN,Done by AIBS and WMU,NaN,Y,2018-06-29 00:00:00,NaN


In [12]:
pickle.dump([df], open('./brain_information.pickle','wb'))

In [13]:
df.to_excel('brain_information.xlsx')

In [27]:
# [ns] = pickle.load(open('./neuron_set_1708cells.pickle', 'rb'))
# metadata = ns.metadata.copy()
# metadata[~metadata['Brain'].isin(idlist)]['Brain'].value_counts()

Series([], Name: Brain, dtype: int64)